In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore, auth
from firebase_admin.firestore import FieldFilter
from google.cloud.firestore_v1.document import DocumentReference
from datetime import datetime
import json 
from dotenv import load_dotenv
from random import randint
import os
import re
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

load_dotenv()

cred = credentials.Certificate(os.getenv("firebase_secret"))
app = firebase_admin.initialize_app(cred)


db = firestore.client()


def load_json_file(filename):
    with open(filename) as fp:
        content = json.load(fp)
    return content


# Carrega os dados que já estão salvos

In [2]:
items = load_json_file("json1123/itens.json")
volumes = load_json_file("json1123/volumes.json")

# Nossos itens que já foram mensurados

In [13]:
itens_mensurados = list(filter(lambda x: re.match(r".*\(AxCxL\)", x["detalhes"]["medidas"]) != None, items.values()))

# Regra 1

Vamos procurar por mobília que tem mais de 1 item

In [16]:
volumes

{'00NODhpg6NKAhZCwTAP1': {'medidas': {'a': 50, 'l': 40, 'c': 60},
  'categoria': 'Mobília',
  'peso': None,
  'items': ['items/W4Vzv5JwwyOrPKJ29wXU'],
  'observacao': '',
  'servicos': [],
  'status': ['Criado'],
  'data_criacao': '2023-10-27T18:30:36.827000+00:00',
  'embalagem': {'caixa': 'caixa_padrao',
   'platico_bolha': None,
   'enchimento': None},
  'responsavel': 'usuarios/carlos.raymundo@ufsc.br',
  'origem': 'ambientes/B119',
  'destino': 'ambientes-norte/B011',
  'propriedades': ['Frágil', 'Proibido Empilhar'],
  'codigo': '00NODhpg6NKAhZCwTAP1',
  'localizacao_atual': 'ambientes/B119',
  'deleted': False},
 '03nNqcfMdK5yANGLNpp8': {'medidas': {'a': 100, 'l': 55, 'c': 55},
  'categoria': 'Mobília',
  'peso': 5,
  'items': ['items/363739'],
  'observacao': '',
  'servicos': [],
  'status': ['Criado'],
  'data_criacao': '2023-09-28T12:46:54.961000+00:00',
  'embalagem': {'caixa': None, 'platico_bolha': None, 'enchimento': None},
  'responsavel': 'usuarios/patricia.barthel@ufs

In [17]:
mobilia_errada = list(filter(lambda x: x["categoria"] == "Mobília" and len(x["items"]) > 1, volumes.values())) 

In [23]:
mobilia_errada_codigo = [x["codigo"] for x in mobilia_errada]

In [33]:
itens_mensurados_volumados_errados = list(filter(lambda x: x["meta"]["volume"] != None and x["meta"]["volume"].split("/")[1] in mobilia_errada_codigo, itens_mensurados))

In [34]:
len(itens_mensurados_volumados_errados)

908

In [35]:
itens_mensurados_volumados_errados

[{'origem': 'A107',
  'edificio': 'BNU02 - Bloco Acadêmico A',
  'short_descricao': 'CADEIRA ESCOLAR',
  'key': 148216,
  'tipo': 'Permanente',
  'ambiente': 'ambientes/A107',
  'meta': {'volume': 'volumes/cute3qhONPhHp3D0rWlh',
   'updated': '2023-09-14T11:40:32+00:00',
   'inteiro': True,
   'volumado': True,
   'partes': []},
  'detalhes': {'patrimonio': 370228,
   'medidas': '89 x 76 x 50 (AxCxL)',
   'valor': 97.99,
   'n_controle': nan,
   'descricao': 'CADEIRA ESCOLAR - C/ASSENTO E ENCOSTO EM POLIPROPILENO NA COR OVO, EM ESTRUTURA DE FERRO NA COR AZUL, C/PRNCHETA FRONTAL MOVEL REVESTIDA DE FORMICA, MED. 89CM DE ALTURA.',
   'peso': '',
   'n_serie': nan,
   'cod_barras': 148216,
   'incorporacao': '2009-09-28T00:00:00+00:00',
   'conservacao': 'EM USO',
   'transf_local': '2018-11-19T00:00:00+00:00',
   'situacao': 'Ativo'},
  'responsavel': 'Rafael dos Reis Abreu'},
 {'origem': 'A104',
  'edificio': 'BNU02 - Bloco Acadêmico A',
  'short_descricao': 'CADEIRA ESCOLAR',
  'key': 1